# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

W. Brandner  ->  W. Brandner  |  ['W. Brandner']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
J. Sauter  ->  J. Sauter  |  ['J. Sauter']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']


Arxiv has 61 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2409.13015
extracting tarball to tmp_2409.13015...

 done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2409.13015/ms.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'tables/phy.tex' from 'tmp_2409.13015/tables/phy.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'tables/joint.tex' from 'tmp_2409.13015/tables/joint.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-package

W. Brandner  ->  W. Brandner  |  ['W. Brandner']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
J. Sauter  ->  J. Sauter  |  ['J. Sauter']


Issues with the citations
list index out of range
Retrieving document from  https://arxiv.org/e-print/2409.13021
extracting tarball to tmp_2409.13021...

 done.
Retrieving document from  https://arxiv.org/e-print/2409.13189


extracting tarball to tmp_2409.13189... done.


/tmp/ipykernel_2752/339448580.py:52: LatexWarning: 2409.13189 did not run properly
bad escape \e at position 36
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.13015-b31b1b.svg)](https://arxiv.org/abs/2409.13015) | **First Resolution of Microlensed Images of a Binary-Lens Event**  |
|| Z. Wu, et al. -- incl., <mark>W. Brandner</mark>, <mark>P. Garcia</mark>, <mark>L. Kreidberg</mark>, <mark>J. Sauter</mark> |
|*Appeared on*| *2024-09-23*|
|*Comments*| *see the ancillary file for animation associated with Fig. 8*|
|**Abstract**|            We resolve the multiple images of the binary-lens microlensing event ASASSN-22av using the GRAVITY instrument of the Very Large Telescope Interferometer (VLTI). The light curves show weak binary perturbations, complicating the analysis, but the joint modeling with the VLTI data breaks several degeneracies, arriving at a strongly favored solution. Thanks to precise measurements of angular Einstein radius \theta_E = 0.726 +/- 0.002 mas and microlens parallax, we determine that the lens system consists of two M dwarfs with masses of M_1 = 0.261 +/- 0.009 M_sun and M_2 = 0.252 +/- 0.017 M_sun, a projected separation of r_\perp = 7.42 +/- 0.33 AU and a distance of D_L = 2.31 +/- 0.09 kpc. The successful VLTI observations of ASASSN-22av open up a new path for studying intermediate-separation (i.e., a few AUs) stellar-mass binaries, including those containing dark compact objects such as neutron stars and stellar-mass black holes.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.13021-b31b1b.svg)](https://arxiv.org/abs/2409.13021) | **Running with the Bulls: The frequency of star-disc encounters in the Taurus star forming region**  |
|| A. J. Winter, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2024-09-23*|
|*Comments*| *20 pages, 20 figures, accepted for publication in A&A*|
|**Abstract**|            Stars and planets form in regions of enhanced stellar density, subjecting protoplanetary discs to gravitational perturbations from neighbouring stars. Observations in the Taurus star-forming have uncovered evidence of at least three recent, star-disc encounters that have truncated discs (HV/DO Tau, RW Aurigae, UX Tau), raising questions about the frequency of such events. We aim to assess the probability of observing truncating star-disc encounters in Taurus. We generate a physically motivated dynamical model including binaries and spatial-kinematic substructure to follow the historical dynamical evolution and stellar encounters in the Taurus star forming region. We track the star-disc encounters and outer disc radius evolution over the lifetime of Taurus. A quarter of discs are truncated below 30 au by dynamical encounters, but this truncation mostly occurs in binaries over the course of a few orbital periods, on a time-scale $\lesssim 0.1$ Myr. Nonetheless, some truncating encounters still occur up to the present age of Taurus. Strongly truncating encounters (ejecting $\gtrsim 10$ percent of the disc mass) occur at a rate $\sim 10$ Myr$^{-1}$, sufficient to explain the encounter between HV and DO Tau $\sim 0.1$ Myr ago. If encounters that eject only $\sim 1$ percent of the disc mass are responsible for RW Aurigae and UX Tau, then they are also expected with encounter rate $\Gamma_\mathrm{enc} \sim 100{-}200$ Myr$^{-1}$. However, the observed sample of recent encounters is probably incomplete, since these examples occurred in systems that are not consistent with random drawing from the mass function. One more observed example would statistically imply additional physics, such as replenishment of the outer disc material.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.13189-b31b1b.svg)](https://arxiv.org/abs/2409.13189) | **Fast Outflow in the Host Galaxy of the Luminous z $=$ 7.5 Quasar J1007$+$2115**  |
|| W. Liu, et al. -- incl., <mark>E. Bañados</mark> |
|*Appeared on*| *2024-09-23*|
|*Comments*| **|
|**Abstract**|            James Webb Space Telescope opens a new window to directly probe luminous quasars powered by billion solar mass black holes in the epoch of reionization and their co-evolution with massive galaxies with unprecedented details. In this paper, we report the first results from the deep NIRSpec integral field spectroscopy study of a quasar at $z = 7.5$. We obtain a bolometric luminosity of $\sim$$1.8\times10^{47}$ erg s$^{-1}$ and a black hole mass of $\sim$0.7--2.5$\times10^{9}$ M$_{\odot}$ based on H$\beta$ emission line from the quasar spectrum. We discover $\sim$2 kpc scale, highly blueshifted ($\sim$$-$870 km/s) and broad ($\sim$1400 km/s) [O III] line emission after the quasar PSF has been subtracted. Such line emission most likely originates from a fast, quasar-driven outflow, the earliest one on galactic-scale known so far. The dynamical properties of this outflow fall within the typical ranges of quasar-driven outflows at lower redshift, and the outflow may be fast enough to reach the circumgalactic medium. Combining both the extended and nuclear outflow together, the mass outflow rate, $\sim$300 M$_{\odot}$yr, is $\sim$60%--380% of the star formation rate of the quasar host galaxy, suggesting that the outflow may expel a significant amount of gas from the inner region of the galaxy. The kinetic energy outflow rate, $\sim$3.6$\times10^{44}$ erg s$^{-1}$, is $\sim$0.2% of the quasar bolometric luminosity, which is comparable to the minimum value required for negative feedback based on simulation predictions. The dynamical timescale of the extended outflow is $\sim$1.7 Myr, consistent with the typical quasar lifetime in this era.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \e at position 36</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2409.13015/figures/fig00.png', 'tmp_2409.13015/figures/fig01.png', 'tmp_2409.13015/figures/fig02.png', 'tmp_2409.13015/figures/fig03.png', 'tmp_2409.13015/figures/fig04.png', 'tmp_2409.13015/figures/fig05.png', 'tmp_2409.13015/figures/fig06.png', 'tmp_2409.13015/figures/fig07.png', 'tmp_2409.13015/figures/lc.png', 'tmp_2409.13015/figures/alpha.png']
copying  tmp_2409.13015/figures/fig00.png to _build/html/
copying  tmp_2409.13015/figures/fig01.png to _build/html/
copying  tmp_2409.13015/figures/fig02.png to _build/html/
copying  tmp_2409.13015/figures/fig03.png to _build/html/
copying  tmp_2409.13015/figures/fig04.png to _build/html/
copying  tmp_2409.13015/figures/fig05.png to _build/html/
copying  tmp_2409.13015/figures/fig06.png to _build/html/
copying  tmp_2409.13015/figures/fig07.png to _build/html/
copying  tmp_2409.13015/figures/lc.png to _build/html/
copying  tmp_2409.13015/figures/alpha.png to _build/html/
exported in  _build/html/2409.13015.md
    + _build

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\au}{{\rm AU}}$
$\newcommand{\sinc}{{\rm sinc}}$
$\newcommand{\kms}{{\rm km} {\rm s}^{-1}}$
$\newcommand{\masyr}{{\rm mas} {\rm yr}^{-1}}$
$\newcommand{\yr}{{\rm yr}}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand{\mas}{{\rm mas}}$
$\newcommand{\sat}{{\rm sat}}$
$\newcommand{\muas}{\mu{\rm as}}$
$\newcommand{\var}{{\rm var}}$
$\newcommand{\pc}{{\rm pc}}$
$\newcommand{\orb}{{\rm orb}}$
$\newcommand{\obs}{{\rm obs}}$
$\newcommand{\max}{{\rm max}}$
$\newcommand{\min}{{\rm min}}$
$\newcommand{\rel}{{\rm rel}}$
$\newcommand{\ast}{{\rm ast}}$
$\newcommand{\eff}{{\rm eff}}$
$\newcommand{\rot}{{\rm rot}}$
$\newcommand{\lsr}{{\rm lsr}}$
$\newcommand{\hel}{{\rm hel}}$
$\newcommand{\geo}{{\rm geo}}$
$\newcommand{\B}{{\rm B}}$
$\newcommand{\Sc}{{\rm S}}$
$\newcommand{\L}{{\rm L}}$
$\newcommand{\E}{{\rm E}}$
$\newcommand{\bpi}{{\bm\pi}}$
$\newcommand{\bmu}{{\bm\mu}}$
$\newcommand{\balpha}{{\bdv\alpha}}$
$\newcommand{\bgamma}{{\bdv\gamma}}$
$\newcommand{\bDelta}{{\bdv\Delta}}$
$\newcommand{\btheta}{{\bdv\theta}}$
$\newcommand{\bphi}{{\bdv\phi}}$
$\newcommand{\bp}{{\bf p}}$
$\newcommand{\bv}{{\bf v}}$
$\newcommand{\bu}{{\bf u}}$
$\newcommand{\deg}{^\circ}$
$\newcommand{\close}{{"close A-"}}$
$\newcommand{\wide}{{"wide A-"}}$
$\newcommand{\revisec}{\color{red}}$</div>



<div id="title">

# First Resolution of Microlensed Images of a Binary-Lens Event

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.13015-b31b1b.svg)](https://arxiv.org/abs/2409.13015)<mark>Appeared on: 2024-09-23</mark> -  _see the ancillary file for animation associated with Fig. 8_

</div>
<div id="authors">

Z. Wu, et al. -- incl., <mark>W. Brandner</mark>, <mark>P. Garcia</mark>, <mark>L. Kreidberg</mark>, <mark>J. Sauter</mark>

</div>
<div id="abstract">

**Abstract:** We resolve the multiple images of the binary-lens microlensing event ASASSN-22av using the GRAVITY instrument of the Very Large Telescope Interferometer (VLTI). The light curves show weak binary perturbations, complicating the analysis, but the joint modeling with the VLTI data breaks several degeneracies, arriving at a strongly favored solution. Thanks to precise measurements of angular Einstein radius $\theta_\E = 0.726\pm0.002$ mas and microlens parallax, we determine that the lens system consists of two M dwarfs with masses of $M_1= 0.261\pm{0.009}  M_\odot$ and $M_2 = 0.252\pm{0.017}  M_\odot$ , a projected separation of $r_\perp = 7.42\pm0.33 \au$ and a distance of $D_\L=2.31\pm0.09$ kpc. The successful VLTI observations of ASASSN-22av open up a new path for studying intermediate-separation (i.e., a few AUs) stellar-mass binaries, including those containing dark compact objects such as neutron stars and stellar-mass black holes.

</div>

<div id="div_fig1">

<img src="tmp_2409.13015/figures/fig00.png" alt="Fig7.1" width="12%"/><img src="tmp_2409.13015/figures/fig01.png" alt="Fig7.2" width="12%"/><img src="tmp_2409.13015/figures/fig02.png" alt="Fig7.3" width="12%"/><img src="tmp_2409.13015/figures/fig03.png" alt="Fig7.4" width="12%"/><img src="tmp_2409.13015/figures/fig04.png" alt="Fig7.5" width="12%"/><img src="tmp_2409.13015/figures/fig05.png" alt="Fig7.6" width="12%"/><img src="tmp_2409.13015/figures/fig06.png" alt="Fig7.7" width="12%"/><img src="tmp_2409.13015/figures/fig07.png" alt="Fig7.8" width="12%"/>

**Figure 7. -** $4\times 2$ panel groups showing the closure phase data and the best-fit microlensing model ($\wide$) for the eight VLTI GRAVITY exposures obtained over three nights. Within each group, the left sub-panel shows the closure phase data as a function of wavelength, with the best-fit model over-plotted as solid lines; the right sub-panel shows the caustics (blue), source trajectory (black line with an arrow) and source position (red circle) at the time of the VLTI observation, with the microlensed images displayed as shaded regions and the critical curves as blue dashed lines. (*fig:cpfit*)

</div>
<div id="div_fig2">

<img src="tmp_2409.13015/figures/lc.png" alt="Fig1" width="100%"/>

**Figure 1. -** Top: The multi-band light curves of the microlensing event ASASSN-22av, which are well described by the best-fit 2L1S model (black solid line) from the joint VLTI and light-curve analysis. The light curves deviate from the best-fit 1L1S model with finite-source effects (black dashed line) near the peak, as shown with the enlarged view in the inset. We analyze three nights of VLTI GRAVITY observations taken after the peak, and their epochs are marked with the red dashed lines.
		Middle: The residuals of the best-fit  2L1S model.
		Bottom: The best-fit 1L1S model shows significant residuals for the $\sim 10$ days around the peak. (*fig:lc*)

</div>
<div id="div_fig3">

<img src="tmp_2409.13015/figures/alpha.png" alt="Fig5" width="100%"/>

**Figure 5. -** The comparisons of $\alpha_{\rm LC}$ derived from light-curve fitting with $\alpha_{\rm VLTI}$ inferred from VLTI data. The colored arcs denote the 1$\sigma$ range of $\alpha_{\rm LC}$ for solutions "close A$+$" (black), "close A$-$" (red), "close B$+$" (cyan), "wide A$+$" (blue), "wide A$-$" (green) and "wide B$+$" (yellow), respectively. The shaded regions indicate the 3$\sigma$ range of $\alpha_{\rm VLTI}$ at $\Psi \sim -100$\deg$, -10$\deg$, 80$\deg$$ and $170$\deg$$. Three solutions (wide A$-$, wide B$+$ and close A$-$) have consistent $\alpha_{\rm LC}$ with $\alpha_{\rm VLTI}$. (*fig:alpha*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.13015"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

188  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
